In [1]:
!pip -q install --upgrade pip 
!pip -q install sagemaker awscli boto3 smdebug pandas matplotlib seaborn --upgrade

#ced mail

     |████████████████████████████████| 1.5MB 3.4MB/s 
     |████████████████████████████████| 303 kB 3.4 MB/s 
     |████████████████████████████████| 3.3 MB 10.8 MB/s 
     |████████████████████████████████| 129 kB 51.2 MB/s 
     |████████████████████████████████| 174 kB 46.4 MB/s 
     |████████████████████████████████| 10.5 MB 20.9 MB/s 
     |████████████████████████████████| 11.6 MB 50.8 MB/s 
     |████████████████████████████████| 283 kB 63.5 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

google-colab 1.0.0 requires pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.2 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you'll h

In [ ]:
# The below code is based on tutorial # https://www.youtube.com/watch?v=sOUhLiI85sU
# continue from 39.05

In [6]:
from IPython.core.display import HTML 


Direct Marketting data sample binary classification


In [3]:
#data 
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip



--2020-09-26 04:37:47--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444572 (434K) [application/x-httpd-php]
Saving to: ‘bank-additional.zip’

bank-additional.zip 100%[===================>] 434.15K  1.63MB/s    in 0.3s    

2020-09-26 04:37:48 (1.63 MB/s) - ‘bank-additional.zip’ saved [444572/444572]

Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/bank-additional/
  inflating: __MACOSX/bank-additional/._.DS_Store  
  inflating: bank-additional/.Rhistory  
  inflating: bank-additional/bank-additional-full.csv  
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: __MACOSX/

In [4]:
!head ./bank-additional/bank-additional-full.csv

"age";"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y"
56;"housemaid";"married";"basic.4y";"no";"no";"no";"telephone";"may";"mon";261;1;999;0;"nonexistent";1.1;93.994;-36.4;4.857;5191;"no"
57;"services";"married";"high.school";"unknown";"no";"no";"telephone";"may";"mon";149;1;999;0;"nonexistent";1.1;93.994;-36.4;4.857;5191;"no"
37;"services";"married";"high.school";"no";"yes";"no";"telephone";"may";"mon";226;1;999;0;"nonexistent";1.1;93.994;-36.4;4.857;5191;"no"
40;"admin.";"married";"basic.6y";"no";"no";"no";"telephone";"may";"mon";151;1;999;0;"nonexistent";1.1;93.994;-36.4;4.857;5191;"no"
56;"services";"married";"high.school";"no";"no";"yes";"telephone";"may";"mon";307;1;999;0;"nonexistent";1.1;93.994;-36.4;4.857;5191;"no"
45;"services";"married";"basic.9y";"unknown";"no";"no";"telephone";"may";"mon";198;1;999;0;"nonex

In [ ]:
# @cl asses binary classification customer will accept or not

In [8]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv("./bank-additional/bank-additional-full.csv",sep = ';')
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',50)
data[:10]




AttributeError: ignored

   age          job  marital            education  default housing loan  \
0   56    housemaid  married             basic.4y       no      no   no   
1   57     services  married          high.school  unknown      no   no   
2   37     services  married          high.school       no     yes   no   
3   40       admin.  married             basic.6y       no      no   no   
4   56     services  married          high.school       no      no  yes   
5   45     services  married             basic.9y  unknown      no   no   
6   59       admin.  married  professional.course       no      no   no   
7   41  blue-collar  married              unknown  unknown      no   no   
8   24   technician   single  professional.course       no     yes   no   
9   25     services   single          high.school       no     yes   no   

     contact month day_of_week  duration  campaign  pdays  previous  \
0  telephone   may         mon       261         1    999         0   
1  telephone   may         mon  

In [10]:
data.shape

(41188, 21)

In [16]:
one_class = data[data['y']=="yes"]
one_class_count = one_class.shape[0]
print("Positive samples :%d" %one_class_count)

zero_class = data[data['y']=="no"]
zero_class_count = zero_class.shape[0]
print("Negative samples :%d" %zero_class_count)

zero_to_one_ratio = zero_class_count/one_class_count 
print("Ratio:%.2f" %zero_to_one_ratio)





Positive samples :4640
Negative samples :36548
Ratio:7.88


<h2> Transformation the dataset  </h2>
Clean up data
when pdays are maximum

In [17]:
[np.min(data['pdays']) ,np.max(data['pdays']) ]

[0, 999]

In [18]:
data['mo_previous_contact'] = np.where(data['pdays'] == 999,1,0)
data = data.drop(['pdays'],axis = 1)

In [19]:
data['job'].value_counts()

admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [20]:
data['not_working'] = np.where(np.in1d(data['job'],['student','retired','unemployed']),1,0  )